# Root Mean Squared Error

In [4]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [5]:
# === 1. Загрузка данных ===
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
# === 2. Разделяем признаки и таргет ===
TARGET = 'exam_score'
ID_COL = 'id'

X = train.drop(columns=[TARGET, ID_COL])
y = train[TARGET]

X_test = test.drop(columns=[ID_COL])

In [7]:
# === 3. Категориальные признаки ===
cat_features = X.select_dtypes(include=['object']).columns.tolist()

In [ ]:
# === Ensemble configs ===
configs = [
    {'depth': 6, 'lr': 0.05},
    {'depth': 8, 'lr': 0.05},
    {'depth': 10, 'lr': 0.05},
    {'depth': 8, 'lr': 0.03},
]

seeds = [0, 42]

test_preds = np.zeros(len(X_test))
model_count = 0

# === Training ensemble ===
for cfg in configs:
    for seed in seeds:
        # ----- normal target -----
        model = CatBoostRegressor(
            loss_function='RMSE',
            iterations=2000,
            learning_rate=cfg['lr'],
            depth=cfg['depth'],
            random_seed=seed,
            verbose=False
        )
        model.fit(X, y, cat_features=cat_features)
        test_preds += model.predict(X_test)
        model_count += 1

        # ----- log target -----
        y_log = np.log1p(y)
        model_log = CatBoostRegressor(
            loss_function='RMSE',
            iterations=2000,
            learning_rate=cfg['lr'],
            depth=cfg['depth'],
            random_seed=seed + 100,
            verbose=False
        )
        model_log.fit(X, y_log, cat_features=cat_features)
        test_preds += np.expm1(model_log.predict(X_test))
        model_count += 1

# === Average predictions ===
test_preds /= model_count

# === Submission ===
submission = pd.DataFrame({
    'id': test['id'],
    'exam_score': test_preds
})

submission.to_csv('submission_ensemble_final.csv', index=False)
print(f'Готово. Использовано моделей: {model_count}')